In [1]:
import torch
import torch.nn.utils.prune as prune 
import torchvision
import onnx
import onnxruntime as ort
import ultralytics
import matplotlib.pyplot as plt
import os
import time
import torch.onnx
from torch.quantization import quantize_dynamic
from ultralytics import YOLO
import cv2
import onnx
from PIL import ImageDraw
import numpy as np
from torchvision import transforms 

In [2]:
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torch2trt import torch2trt

In [3]:
# Check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
providers = ["TensorrtExecutionProvider", "CUDAExecutionProvider"]

In [5]:
# Load the ONNX model
model_path = 'yolov8s_quantized.onnx'
session = ort.InferenceSession(model_path,provider=providers)

In [6]:
# Prepare the input data
input_name = session.get_inputs()[0].name
input_shape = session.get_inputs()[0].shape

In [7]:
# Print the expected input shape
print("Expected input shape:", input_shape)

Expected input shape: ['batch', 3, 'height', 'width']


In [8]:
img = Image.open("bus.jpg")
img_width, img_height = img.size;
img = img.resize((640,640))

In [9]:
# Load and preprocess the image
img_path = 'bus.jpg'

# Load image using OpenCV
img = cv2.imread(img_path)

# Resize to the expected dimensions
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
img = cv2.resize(img, (input_shape[3], input_shape[2]))  # Resize to (640, 640)

# Normalize the image if required (depending on your model's preprocessing)
img = img.astype(np.float32) / 255.0

# Add batch dimension
img = np.expand_dims(img, axis=0)

# Reorder dimensions from (1, height, width, channels) to (1, channels, height, width)
img = np.transpose(img, (0, 3, 1, 2))

# Check the shape of the input
print("Input shape after reordering:", img.shape)
num_runs = 100
for _ in range(num_runs):
# Run inference
    st = time.perf_counter()
    outputs = session.run(None, {input_name: img})
    et = time.perf_counter() - st
    print(f"FPS {1/et} {et}")
# Process the output (this depends on your specific model)
print("Model outputs:", outputs)

error: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'resize'
> Overload resolution failed:
>  - Can't parse 'dsize'. Sequence item with index 0 has a wrong type
>  - Can't parse 'dsize'. Sequence item with index 0 has a wrong type


In [ ]:
output = outputs[0]
output.shape

In [ ]:
output = output[0]
output.shape

In [ ]:
output = output.transpose()

In [ ]:
output.shape

In [ ]:
len(outputs)

In [ ]:
row = output[0]
print(row)

In [ ]:
xc,yc,w,h = row[:4]

In [ ]:
x1 = xc-w/2
y1 = yc-h/2
x2 = xc+w/2
y2 = yc+h/2

In [ ]:
x1 = (xc - w/2) / 640 * img_width
y1 = (yc - h/2) / 640 * img_height
x2 = (xc + w/2) / 640 * img_width
y2 = (yc + h/2) / 640 * img_height

In [ ]:
prob = row[4:].max()
class_id = row[4:].argmax()

print(prob, class_id)

In [ ]:
# Array of YOLOv8 class labels
yolo_classes = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", "dog", "horse",
    "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella", "handbag", "tie",
    "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove",
    "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon",
    "bowl", "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut",
    "cake", "chair", "couch", "potted plant", "bed", "dining table", "toilet", "tv", "laptop", "mouse",
    "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "book",
    "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
]

In [ ]:
label = yolo_classes[class_id]

In [ ]:
def parse_row(row):
    xc,yc,w,h = row[:4]
    x1 = (xc-w/2)#/640*img_width
    y1 = (yc-h/2)#/640*img_height
    x2 = (xc+w/2)#/640*img_width
    y2 = (yc+h/2)#/640*img_height
    prob = row[4:].max()
    class_id = row[4:].argmax()
    label = yolo_classes[class_id]
    return [x1,y1,x2,y2,label,prob]

In [ ]:
boxes = [row for row in [parse_row(row) for row in output] if row[5]>0.5]
len(boxes)

In [ ]:
[parse_row(row) for row in output]

In [ ]:
img_path = 'bus.jpg'
img = Image.open(img_path)
print(img.size)
img = img.resize((640,640))
print(img.size)
draw = ImageDraw.Draw(img)

In [ ]:
for box in boxes:
    x1,y1,x2,y2,class_id,prob = box
    print(x1,y1,x2,y2,class_id,prob)
    draw.rectangle((int(x1),int(y1),int(x2),int(y2)),None,"#00ff00")

img